In [30]:
import os, cv2, torch, json, wandb, argparse
import numpy as np
import ultralytics.nn.tasks as tasks
import ultralytics.utils.torch_utils as torch_utils
from pathlib import Path
from ultralytics import YOLO
from ultralytics.utils import ops
from ultralytics.utils.metrics import ap_per_class
from utils import bounding_boxes, FisheyeDetectionValidator
from ultralytics.models.yolo.detect.train import DetectionTrainer
from yolov8_monkey_patches import load_model_custom, parse_dcn_model, get_flops_pass

In [6]:
if __name__ == "__main__":
  # monkey patches
  DetectionTrainer.get_model = load_model_custom
  tasks.parse_model = parse_dcn_model
  torch_utils.get_flops = get_flops_pass

  # parser = argparse.ArgumentParser(description="yolov8x fisheye experiment")
  # parser.add_argument('-conf', type=float, default=0.001, help="batch size")
  # parser.add_argument('-iou',  type=float, default=0.5,  help="number of workers")
  # parser.add_argument('-model', type=str, default="yolov8x_dcn.yaml", help="batch size")
  # args = parser.parse_args()

  config = {"conf": 0.001,
            "iou" : 0.5}
  
  run = wandb.init(project="fisheye-challenge", name="yolov8x-dcn-lr2e-5_eval", config=config)

  art = run.use_artifact("g1y5x3/fisheye-challenge/run_u5sa7jfr_model:v0")
  art_dir = art.download()

  data_dir = "/workspace/FishEye8k/dataset/Fisheye8K_all_including_train/test/images/"

  with open("/workspace/FishEye8k/dataset/Fisheye8K_all_including_train/test/test.json") as f:
    images = json.load(f)
  files = images["images"]
  sources = [data_dir+img["file_name"] for img in files]
  print(f"Total data for inference {len(sources)}")

  model = YOLO(f"{art_dir}/best.pt") # model was trained on COCO dataset

  result_json = []
  for i in range(len(sources)//128+1):
    start = i*128
    end = (i+1)*128 if i <= 20 else -1 

    results = model.predict(sources[start:end], imgsz=1280, 
                            conf=config["conf"], iou=config["iou"], 
                            stream=False, verbose=True)

    for result in results:
      image_id = result.path.rsplit('/',1)[-1]
      bboxes = ops.xyxy2ltwh(result.boxes.xyxy.cpu().numpy())
      conf = result.boxes.conf.cpu().numpy()
      cls = result.boxes.cls.cpu().numpy()
      for cat, score, box in zip(cls, conf, bboxes.tolist()):
        result_json.append(
          {
            "image_id": image_id,
            "category_id": int(cat),
            "bbox": [float(x) for x in box],
            "score": float(score)
          }
        )

  with open("results/yolov8x_eval_fisheye.json", "w") as f:
    json.dump(result_json, f)

  table = wandb.Table(columns=["ID", "Image"])

  # Initialize the result calculation
  class_name = {0: 'bus', 1: 'bike', 2: 'car', 3: 'pedestrian', 4: 'truck'}
  fisheye_eval = FisheyeDetectionValidator()
  fisheye_eval.init_metrics(class_name)

  # Load ground truth and predictions
  gt_dir   = "/workspace/FishEye8k/dataset/Fisheye8K_all_including_train/test/test.json"
  with open(gt_dir) as f: gts = json.load(f)
  gt_img, gt_ann = gts["images"], gts["annotations"]

  image_list = [img["file_name"] for img in gt_img]

  pred_dir = "results/yolov8x_eval_fisheye.json"
  with open(pred_dir) as f: preds = json.load(f)

  for img_name in image_list:
    print(img_name)
    # get ground truth
    img_id = [gt["id"] for gt in gt_img if gt["file_name"] == img_name]
    bboxes = np.array([gt["bbox"] for gt in gt_ann if gt["image_id"] == img_id[0]])
    bboxes = ops.ltwh2xyxy(bboxes)
    cls = np.array([gt["category_id"] for gt in gt_ann if gt["image_id"] == img_id[0]])
    gt_array = np.concatenate((bboxes, cls[:, np.newaxis]), axis=1)
    
    # get predictions
    bboxes = np.array([pred["bbox"] for pred in preds if pred["image_id"] == img_name])
    if len(bboxes) > 0:
      bboxes = ops.ltwh2xyxy(bboxes)
      confs = np.array([pred["score"] for pred in preds if pred["image_id"] == img_name])
      cls = np.array([pred["category_id"] for pred in preds if pred["image_id"] == img_name])
      pred_array = np.concatenate((bboxes, confs[:, np.newaxis], cls[:, np.newaxis]), axis=1)
    else:
      pred_array = np.array([])

    # img = cv2.imread(data_dir + img_name)
    # box_img = bounding_boxes(img, pred_array.tolist(), gt_array.tolist(), class_name)
    # table.add_data(img_name, box_img)

    fisheye_eval.update_metrics([torch.tensor(pred_array)], [torch.tensor(gt_array)])


wandb: Currently logged in as: g1y5x3. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact run_u5sa7jfr_model:v0, 130.58MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.3


Total data for inference 2712

0: 1280x1280 2 buss, 18 bikes, 16 cars, 13 pedestrians, 28.3ms
1: 1280x1280 12 bikes, 8 cars, 17 pedestrians, 2 trucks, 28.3ms
2: 1280x1280 4 buss, 99 bikes, 58 cars, 48 pedestrians, 1 truck, 28.3ms
3: 1280x1280 1 bus, 43 bikes, 33 cars, 24 pedestrians, 1 truck, 28.3ms
4: 1280x1280 5 buss, 33 bikes, 21 cars, 34 pedestrians, 4 trucks, 28.3ms
5: 1280x1280 6 buss, 59 bikes, 33 cars, 45 pedestrians, 28.3ms
6: 1280x1280 6 buss, 43 bikes, 17 cars, 22 pedestrians, 2 trucks, 28.3ms
7: 1280x1280 4 buss, 50 bikes, 24 cars, 26 pedestrians, 1 truck, 28.3ms
8: 1280x1280 1 bus, 93 bikes, 33 cars, 32 pedestrians, 1 truck, 28.3ms
9: 1280x1280 4 buss, 31 bikes, 14 cars, 16 pedestrians, 28.3ms
10: 1280x1280 5 buss, 20 bikes, 20 cars, 18 pedestrians, 28.3ms
11: 1280x1280 6 buss, 44 bikes, 20 cars, 48 pedestrians, 1 truck, 28.3ms
12: 1280x1280 5 buss, 17 bikes, 21 cars, 9 pedestrians, 2 trucks, 28.3ms
13: 1280x1280 2 buss, 12 bikes, 17 cars, 8 pedestrians, 2 trucks, 28.3ms
1

In [26]:
tp = np.concatenate([tp.numpy() for tp in fisheye_eval.stats['tp']], axis=0)
conf = np.concatenate([conf.numpy() for conf in fisheye_eval.stats['conf']], axis=0)
pred_cls = np.concatenate([pred_cls.numpy() for pred_cls in fisheye_eval.stats['pred_cls']], axis=0)
target_cls = np.concatenate([target_cls.numpy() for target_cls in fisheye_eval.stats['target_cls']], axis=0)

array([2, 3, 1, ..., 2, 2, 3])

In [8]:
from ultralytics.utils.metrics import ap_per_class

In [32]:
ap_per_class(tp, conf, pred_cls, target_cls, plot=True, save_dir=Path("results"), names=class_name)

(array([        607,       13296,       10376,        1114,         449]),
 array([        101,        4485,        1971,        1104,         213]),
 array([    0.85779,     0.74777,     0.84034,     0.50218,     0.67866]),
 array([    0.65269,     0.50613,     0.73521,     0.42297,     0.37448]),
 array([    0.74131,     0.60367,     0.78427,     0.45918,     0.48264]),
 array([[    0.73731,     0.72665,     0.71801,     0.69913,     0.66766,     0.61699,     0.52914,     0.34616,     0.15361,    0.013309],
        [    0.68442,     0.63373,     0.56644,     0.47972,     0.38096,     0.27288,     0.16101,    0.065412,    0.012519,  0.00061003],
        [    0.82839,     0.80346,     0.76197,     0.70163,     0.62988,     0.53592,     0.42657,     0.27437,     0.10809,    0.012981],
        [    0.42199,     0.40306,     0.37339,     0.32341,     0.23886,     0.14751,    0.069896,    0.024575,   0.0022307,  0.00030505],
        [    0.48433,     0.46276,     0.44667,     0.42577,     